In [10]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV  # Perforing grid search
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

In [2]:
train_data = pd.read_csv('Traindata_cat3.csv')

In [3]:
train_y = train_data.pop('click').values.astype('int')   # target variable
train_data.pop('id')
col = train_data.columns
train_x = train_data[col]

train_x.iloc[:,0:23] = train_x.iloc[:,0:23].astype('str')
#train_x = train_x.apply(lambda x: pd.factorize(x)[0])
#train_x = train_x.astype('category')

train_x.columns

Index(['hour', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category',
       'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip',
       'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16',
       'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [4]:
train_x.head()

,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,11,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,...,1,0,-1,320,50,1872,3,39,-1,23
1,23,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,...,1,0,15707,320,50,1722,0,35,100083,79
2,14,1005,0,85f751fd,c4e18dd6,50e219e0,0,0,0f2161f8,a99f214a,...,1,2,-1,320,50,-1,3,39,-1,23
3,11,1005,0,85f751fd,c4e18dd6,50e219e0,0,0,0f2161f8,a99f214a,...,1,0,-1,320,50,-1,3,171,100228,156
4,2,1005,0,85f751fd,c4e18dd6,50e219e0,0,0,0f2161f8,a99f214a,...,1,0,-1,320,50,1887,3,39,-1,23


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
# prepare input data
def prepare_inputs(X_train):
    ohe = OneHotEncoder()
    ohe.fit(X_train)
    X_train_enc = ohe.transform(X_train)
    return X_train_enc

In [6]:
x_train = prepare_inputs(train_x)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(x_train, train_y, test_size=0.3, random_state=42)

In [11]:
# a 0.407 model

from sklearn import metrics
from xgboost.sklearn import XGBClassifier
clf = XGBClassifier(
silent=0,
#nthread=4,# cpu 
learning_rate= 0.3,
min_child_weight=1, 
max_depth=6, 
gamma=0, 
subsample=1, 
max_delta_step=0,
colsample_bytree=1, 
reg_lambda=1,  
#reg_alpha=0, # L1 
#scale_pos_weight=1, 
#objective= 'multi:softmax', 
#num_class=10, 
n_estimators=100, # tree number
seed=1000 # random seed
#eval_metric= 'logloss'
)
clf.fit(X_train, y_train,eval_set=[(X_train, y_train)],eval_metric='logloss',verbose=False)
evals_result = clf.evals_result()
y_true, y_pred = y_val, clf.predict_proba(X_val)[:, 1]
print("logloss : %0.4f" % log_loss(y_true,y_pred))

logloss : 0.4070


In [ ]:
# hyperparameter tuning, put whatever you want to try in the for loop, remember to change print

for s in [0.9]:
    for n in [150,200,250]:
        clf = XGBClassifier(
              silent=0, 
              #nthread=4,
              learning_rate= 0.1,
              min_child_weight=0.5, 
              max_depth=5, 
              gamma=0.1, 
              subsample=1, 
              max_delta_step=0,
              colsample_bytree=1, 
              reg_lambda=1,
              #reg_alpha=1, # L1 
              #scale_pos_weight=1,
              #objective= 'multi:softmax',
              #num_class=10,
              n_estimators=n, 
              seed=1000
              #eval_metric= 'logloss'
              )
        clf.fit(X_train,y_train,eval_set=[(X_train, y_train)],eval_metric='logloss',verbose=False)
        #evals_result = clf.evals_result()
        y_true, y_pred = y_val, clf.predict_proba(X_val)[:, 1]
        print("Logloss: %0.3f, subsample: %0.1f,n: %0.1f "%(log_loss(y_true,y_pred),s,n))